In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

In [2]:
## df = pd.read_csv('../constants/state_fips.csv')
## df.rename({'FIPS':'Fips'},axis=1,inplace=True)
## df.to_csv('state_fips.csv',index=False)

# df = pd.read_csv('state_fips.csv',index_col='Fips')
# df.head()

In [3]:
#####################################################
## COVID HUB ENSEMBLE STATE PREDICTIONS ############
#####################################################

import glob
import os
from tqdm.notebook import tqdm 

my_dir = os.path.join(DATA_DIR,'raw','COVIDhub-ensemble')
files = glob.glob(f'{my_dir}/*.csv')

types = ['point']
targets = ['1 wk ahead inc death']

all_dfs = []
for f in tqdm(files,total=len(files)):
    df = pd.read_csv(f)
    ## Choose which types (only point for now)
    df = df[df.type=='point']
    ## Only 1 wk ahead inc
    df = df[df.target.isin(targets)]
    all_dfs.append(df)
df_all = pd.concat(all_dfs)

## REFORMAT dataframe ...
## choose cols
df = df_all[['location','target_end_date','value']]
## rename cols
df.rename({'location':'Fips',
           'target_end_date':'Date',
           'value':'Pred_state_deaths'}, axis=1, inplace=True)
## dtype check: good

## SAVE CSV!
fpath = os.path.join(DATA_DIR,'processed',f'formatted_COVIDhub-ensemble_{DT}.csv')
df.to_csv(fpath,index=False)

  0%|          | 0/210 [00:00<?, ?it/s]

In [4]:
#####################################################
## NYT TRUE COUNTY DEATHS ###########################
#####################################################

fpath = os.path.join(DATA_DIR,'raw','nytimes','us-counties.csv')
df = pd.read_csv(fpath)
## drop nans lmao.
if df.isnull().values.any()==True:
    print('Dropped NaNs!')
    df.dropna(inplace=True)
    
## capitalize cols.
df.columns = df.columns.str.capitalize()

## IMPORTANT: pull out samples from 'nytimes' that have matched dates to 'COVIDhub-ensemble' ...
df_hub,_ = load_csv('formatted_COVIDhub-ensemble')
my_dates = df_hub.Date.unique().tolist()
df = df[df.Date.isin(my_dates)]

## REFORMAT dataframe ...
## choose cols
df = df[['Date','Fips','Deaths']]
## rename cols
df.rename({'Deaths':'True_county_deaths'},axis=1,inplace=True)
## fix dtypes
df['Fips'] = df.Fips.astype('int64').astype('str')

## SAVE CSV!
fpath = os.path.join(DATA_DIR,'processed',f'formatted_nytimes-us-counties_{DT}.csv')
df.to_csv(fpath,index=False)

Dropped NaNs!


In [5]:
#####################################################
## FIPS DICTIONARY  #################################
#####################################################

In [6]:
df = pd.read_csv('../constants/state_fips.csv')
df

# df.rename({'FIPS':'Fips'},axis=1,inplace=True)
# df.to_csv('state_fips.csv',index=False)

# df = pd.read_csv('state_fips.csv',index_col='Fips')
# df.head()

,State,Postal,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6
5,Colorado,CO,8
6,Connecticut,CT,9
7,Delaware,DE,10
8,District of Columbia,DC,11
9,Florida,FL,12


In [ ]:
## SAVE CSV!
fpath = os.path.join(DATA_DIR,'processed',f'formatted_nytimes-us-counties_{DT}.csv')
df.to_csv(fpath,index=False)

In [ ]:

df

In [ ]:
df.dtypes